In [3]:
from trackml.dataset import load_event
from trackml.utils import add_position_quantities, add_momentum_quantities, decode_particle_id

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

path = '/nfs/fanae/user/uo290313/Desktop/TFG/dataset/trackml-particle-identification/train_1/'
event = 'event000001000'

print('Loading event {}'.format(path+event))

hits, cells, particles, truth = load_event(path+event)

mem_bytes = (hits.memory_usage(index=True).sum() 
             + cells.memory_usage(index=True).sum() 
             + particles.memory_usage(index=True).sum() 
             + truth.memory_usage(index=True).sum())
print('{} memory usage {:.2f} MB'.format(event, mem_bytes / 2**20))

Loading event /nfs/fanae/user/uo290313/Desktop/TFG/dataset/trackml-particle-identification/train_1/event000001000
event000001000 memory usage 18.46 MB


In [ ]:
'''
Representación de los hits
'''
print(hits.head())

g = sns.jointplot(x=hits.x, y=hits.y, s=1, height=12)
g.ax_joint.cla()
plt.sca(g.ax_joint)

volumes = hits.volume_id.unique()
for volume in volumes:
    v = hits[hits.volume_id == volume]
    plt.scatter(v.x, v.y, s=3, label='volume {}'.format(volume))

plt.xlabel('X (mm)')
plt.ylabel('Y (mm)')
plt.legend()
plt.show()

In [24]:
from kalman_filter import KalmanFilter

#print(particles.head())

particles = decode_particle_id(particles)
particles = add_momentum_quantities(particles)

x = [particles.vx, particles.vy, particles.vz, particles.px, particles.py, particles.pz]    # La dimensión de los datos será de 6
print(particles.head())

''' Suposiciones de matrices de Kalman Filter '''
# Suponemos que el sistema es lineal y que la velocidad es constante
# El sistema se modela con las siguientes matrices

A = np.array([[1, 0, 0, 1, 0, 0],   # Establecemos que la posición se actualiza con la velocidad
              [0, 1, 0, 0, 1, 0],
              [0, 0, 1, 0, 0, 1],
              [0, 0, 0, 1, 0, 0],   # El estado de la velocidad se propaga
              [0, 0, 0, 0, 1, 0],
              [0, 0, 0, 0, 0, 1]])

C = np.eye(6)  # Covarianza inicial (suponemos que comenzamos con incertidumbre en el estado)
F = A  # La propagación es igual a la matriz de transición de estado A
H = np.array([[1, 0, 0, 0, 0, 0],  # Solo medimos las posiciones x, y, z
              [0, 1, 0, 0, 0, 0],
              [0, 0, 1, 0, 0, 0]])

Q = np.eye(6) * 0.01  # Ruido de proceso (ajusta según el sistema)
R = np.array([[0.1, 0, 0],  # Ruido de medición (ajusta según el sensor)
              [0, 0.1, 0],
              [0, 0, 0.1]])

# Estado inicial (suponemos que inicialmente la partícula está en la posición 0, 0, 0 y tiene velocidad 0)
x0 = np.array([0, 0, 0, 0, 0, 0])

# Crear el filtro de Kalman
kf = KalmanFilter(A, C, F, H, Q, R, x0)

# Lista para almacenar las estimaciones de posición y velocidad
estimaciones = []

# Iterar sobre el dataset
for index, row in particles.iterrows():
    # Las mediciones son las componentes px, py, pz (que son las posiciones que tenemos)
    m = np.array([[row['px']], [row['py']], [row['pz']]])  # Convertir la medición a un array para el filtro
    kf.predict()  # Paso de predicción
    estimacion = kf.update(m)  # Paso de actualización con la medición actual
    estimaciones.append(estimacion)

# Convertir las estimaciones en un DataFrame para analizar
estimaciones_df = pd.DataFrame(estimaciones, columns=['pos_x', 'pos_y', 'pos_z', 'vel_x', 'vel_y', 'vel_z'])

# Mostrar las estimaciones
particles['pos_x_estimada'] = estimaciones_df['pos_x']
particles['pos_y_estimada'] = estimaciones_df['pos_y']
particles['pos_z_estimada'] = estimaciones_df['pos_z']
particles['vel_x_estimada'] = estimaciones_df['vel_x']
particles['vel_y_estimada'] = estimaciones_df['vel_y']
particles['vel_z_estimada'] = estimaciones_df['vel_z']

print(particles)



        particle_id        vx        vy        vz         px         py  \
0  4503668346847232 -0.009288  0.009861 -0.077879  -0.055269   0.323272   
1  4503737066323968 -0.009288  0.009861 -0.077879  -0.948125   0.470892   
2  4503805785800704 -0.009288  0.009861 -0.077879  -0.886484   0.105749   
3  4503874505277440 -0.009288  0.009861 -0.077879   0.257539  -0.676718   
4  4503943224754176 -0.009288  0.009861 -0.077879  16.439400 -15.548900   

          pz  q  nhits  vertex_id  primary_id  generation  secondary_id  \
0  -0.203492 -1      8          1           1           0             0   
1   2.010060  1     11          1           2           0             0   
2   0.683881 -1      0          1           3           0             0   
3   0.991616  1     12          1           4           0             0   
4 -39.824902  1      3          1           5           0             0   

   process         pt      pphi      peta          p  
0        0   0.327963  1.740126 -0.586301  

ValueError: operands could not be broadcast together with shapes (6,) (6,3) 